In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

data1 = "I'm designing a document and don't want to get bogged down in what the text actually says"
data2 = "I'm creating a template for various paragraph styles and need to see what they will look like."
data3 = "I'm trying to learn more about some features of Microsoft Word and don't want to practice on a real document"

df1 = pd.DataFrame({'First_Para':[data1], 'Second_Para':[data2], 'Third_Para':[data3]})
vectorizer = TfidfVectorizer()
doc_vec = vectorizer.fit_transform(df1.iloc[0])

df2 = pd.DataFrame(doc_vec.toarray().transpose(),index=vectorizer.get_feature_names_out())

df2.columns = df1.columns
print(df2)

           First_Para  Second_Para  Third_Para
about        0.000000     0.000000    0.254170
actually     0.288540     0.000000    0.000000
and          0.170416     0.162095    0.150117
bogged       0.288540     0.000000    0.000000
creating     0.000000     0.274451    0.000000
designing    0.288540     0.000000    0.000000
document     0.219442     0.000000    0.193303
don          0.219442     0.000000    0.193303
down         0.288540     0.000000    0.000000
features     0.000000     0.000000    0.254170
for          0.000000     0.274451    0.000000
get          0.288540     0.000000    0.000000
in           0.288540     0.000000    0.000000
learn        0.000000     0.000000    0.254170
like         0.000000     0.274451    0.000000
look         0.000000     0.274451    0.000000
microsoft    0.000000     0.000000    0.254170
more         0.000000     0.000000    0.254170
need         0.000000     0.274451    0.000000
of           0.000000     0.000000    0.254170
on           

## BM25

In [9]:
from rank_bm25 import BM25Okapi

corpus=[
    "I will take the ring, though I do not know the way.",
    "I will help you bear this burden, Frodo Baggins, as long as it is yours to bear",
    "If by my life or death I can protect you, I will."
]

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [11]:
query="I will take"
tokenized_query = query.split(" ")
doc_scores = bm25.get_scores(tokenized_query)
print(doc_scores)

[0.79842666 0.18266174 0.15070007]


## LDA

In [12]:
# Importing modules
import pandas as pd
# Read Corona Tweets
tweets = pd.read_csv(r"C:\Users\liuru\Desktop\EE6405\Data\archive\Corona_NLP_train.csv", encoding='latin-1')
# Print head
tweets.head()

,ï»¿UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [13]:
# Remove the columns
tweets = tweets.drop(columns=['ï»¿UserName', 'ScreenName', 'Location','TweetAt','Sentiment'], axis=1).sample(100)
# Print out the first rows of papers
tweets.head()

,OriginalTweet
29114,New York is paying absurd prices for COVID 19 ...
34458,SCAM ALERT Be cautious of anyone selling prod...
35785,"Since the end of March, a US #supermarket has ..."
6118,Thank you to the medical staff supermarket emp...
14877,Wonderful! Greed of AirBNB landlords is futile...


In [14]:
# Load the regular expression library
import re
# Remove punctuation
tweets['OriginalTweet_processed'] = \
tweets['OriginalTweet'].map(lambda x: re.sub('[@#,\.!?]', '', x))
# Convert the tweets to lowercase
tweets['OriginalTweet_processed'] = \
tweets['OriginalTweet_processed'].map(lambda x: x.lower())
# Print out the first rows of tweets
tweets['OriginalTweet_processed'].head()

29114    new york is paying absurd prices for covid 19 ...
34458    scam alert  be cautious of anyone selling prod...
35785    since the end of march a us supermarket has de...
6118     thank you to the medical staff supermarket emp...
14877    wonderful greed of airbnb landlords is futile ...
Name: OriginalTweet_processed, dtype: object

In [15]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['https', 'tco'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = tweets.OriginalTweet_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['new', 'york', 'paying', 'absurd', 'prices', 'covid', 'supplies', 'ray', 'machine', 'price', 'gouging', 'laws', 'usually', 'apply', 'situation', 'great', 'working']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\liuru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1)]


In [17]:
from pprint import pprint
# number of topics
num_topics = 7
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.019*"coronavirus" + 0.011*"covid" + 0.011*"supermarket" + '
  '0.008*"customers" + 0.007*"hours" + 0.007*"number" + 0.006*"enter" + '
  '0.006*"retail" + 0.006*"employees" + 0.006*"march"'),
 (1,
  '0.013*"covid" + 0.012*"store" + 0.009*"grocery" + 0.009*"supermarket" + '
  '0.009*"food" + 0.009*"coronavirus" + 0.008*"amp" + 0.007*"dona" + '
  '0.007*"go" + 0.007*"online"'),
 (2,
  '0.019*"covid" + 0.012*"amp" + 0.010*"says" + 0.009*"food" + 0.009*"panic" + '
  '0.007*"one" + 0.006*"buying" + 0.006*"supermarket" + 0.006*"people" + '
  '0.006*"spread"'),
 (3,
  '0.020*"coronavirus" + 0.013*"covid" + 0.010*"prices" + 0.009*"grocery" + '
  '0.008*"covid_" + 0.008*"store" + 0.006*"sanitizer" + 0.006*"like" + '
  '0.006*"dutch" + 0.006*"mossad"'),
 (4,
  '0.012*"amp" + 0.012*"coronavirus" + 0.010*"supermarket" + 0.010*"workers" + '
  '0.010*"covid" + 0.007*"us" + 0.007*"prices" + 0.007*"avoid" + 0.007*"work" '
  '+ 0.006*"store"'),
 (5,
  '0.032*"covid" + 0.015*"coronavirus" + 0.0

In [18]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./results/ldavis_prepared_'+str(num_topics))
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.125965 -0.005411       1        1  20.769981
1      0.049636 -0.035202       2        1  18.608961
6      0.011726 -0.116838       3        1  16.797752
4      0.062010  0.032588       4        1  13.295292
0      0.020769  0.050453       5        1  11.582716
3      0.006295  0.038616       6        1   9.684018
2     -0.024472  0.035794       7        1   9.261280, topic_info=            Term       Freq      Total Category  logprob  loglift
241         says   2.000000   2.000000  Default  30.0000  30.0000
53           amp  11.000000  11.000000  Default  29.0000  29.0000
34   coronavirus  25.000000  25.000000  Default  28.0000  28.0000
73       workers   4.000000   4.000000  Default  27.0000  27.0000
307    customers   2.000000   2.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
502    difficult   0.620690   1.777122   Topic7  -5.6414   1.3274
343         life   0.620682   1.870107   Topic7  -5.6414   1.2764
117   everything   0.620673   2.476503   Topic7  -5.6414   0.9955
131          nhs   0.620662   2.415663   Topic7  -5.6414   1.0204
272          get   0.620650   2.961065   Topic7  -5.6415   0.8168

[432 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
103       6  0.903576    abandoned
1016      3  0.764798   absolutely
900       2  0.807752  acmemarkets
197       2  0.807759        allow
448       4  0.343841         also
...     ...       ...          ...
113       6  0.903584          xcl
1001      5  0.835312     xudrvoxz
805       1  0.974327           ya
672       6  0.903587           yg
698       7  0.865470   ynmuccdbwd

[536 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 2, 7, 5, 1, 4, 3])

## LSA

In [19]:
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

In [21]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['https', 'tco'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
def stem_words(texts):
    return [[p_stemmer.stem(word) for word in simple_preprocess(str(doc))
             ] for doc in texts]

data = tweets.OriginalTweet_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['new', 'york', 'paying', 'absurd', 'prices', 'covid', 'supplies', 'ray', 'machine', 'price', 'gouging', 'laws', 'usually', 'apply', 'situation', 'great', 'working']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\liuru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
doc_term_matrix = [id2word.doc2bow(twt) for twt in data_words]

In [24]:
lsa_model = LsiModel(doc_term_matrix, num_topics=num_topics, id2word = id2word)
print(lsa_model.print_topics(num_topics=num_topics, num_words=10))

[(0, '0.567*"covid" + 0.345*"coronavirus" + 0.201*"store" + 0.191*"food" + 0.182*"grocery" + 0.169*"supermarket" + 0.169*"amp" + 0.158*"people" + 0.139*"shopping" + 0.113*"go"'), (1, '0.461*"store" + 0.457*"grocery" + -0.307*"amp" + -0.269*"covid" + -0.179*"food" + 0.165*"covid_" + 0.158*"people" + -0.115*"supermarket" + -0.092*"online" + 0.081*"anyonea"'), (2, '0.521*"coronavirus" + -0.330*"food" + -0.281*"amp" + -0.180*"people" + -0.159*"store" + -0.158*"grocery" + -0.152*"dona" + 0.142*"sanitizer" + -0.142*"go" + 0.112*"hand"'), (3, '0.453*"amp" + -0.310*"covid" + 0.190*"coronavirus" + -0.189*"people" + -0.173*"dona" + 0.141*"grocery" + -0.139*"cana" + 0.132*"food" + -0.131*"stock" + 0.129*"supermarket"'), (4, '-0.289*"people" + 0.245*"shopping" + 0.245*"covid" + -0.212*"dona" + -0.187*"coronavirus" + 0.184*"store" + -0.175*"cana" + 0.170*"online" + -0.156*"go" + 0.153*"grocery"'), (5, '0.399*"supermarket" + -0.265*"food" + 0.202*"staff" + 0.198*"go" + -0.191*"coronavirus" + 0.158*"

C:\Users\liuru\anaconda3\envs\tensorflow_base\lib\site-packages\gensim\models\lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(


## NMF

In [25]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
 
tweets = pd.read_csv(r"C:\Users\liuru\Desktop\EE6405\Data\archive\Corona_NLP_train.csv", encoding='latin-1')
# Print head
tweets.head()

,ï»¿UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [26]:
# Remove the columns
tweets = tweets.drop(columns=['ï»¿UserName', 'ScreenName', 'Location','TweetAt','Sentiment'], axis=1).sample(100)
# Print out the first rows of papers
tweets.head()

,OriginalTweet
23126,@TwixM6 @RedMajid @tanveer_khatana @MurtazaVie...
32978,When #COVID19 turns #toiletpaper into #mission...
15735,From support supermarket only.\r\r\n@fashionis...
38043,Finding Ways To Work things Out During Isolati...
31009,"""I think I was trying to suggest something abo..."


In [27]:
# use tfidf by removing tokens that don't appear in at least 50 documents
vect = TfidfVectorizer(min_df=10, stop_words=stop_words )
 
# Fit and transform
X = vect.fit_transform(tweets.OriginalTweet)

In [28]:
# Create an NMF instance: model
# the 10 components will be the topics
model = NMF(n_components=10, random_state=5)
 
# Fit the model to TF-IDF
model.fit(X)
 
# Transform the TF-IDF: nmf_features
nmf_features = model.transform(X)

In [29]:
components_df = pd.DataFrame(model.components_, columns=vect.get_feature_names_out())
components_df

,19,co,consumer,coronavirus,covid,covid19,covid_19,grocery,like,need,prices,shopping,store,supermarket
0,0.000000,8.395243,0.000000,0.000000,0.211958,0.000000,0.000000,0.000000,0.000000,0.001831,0.071069,0.045082,0.000000,0.000000
1,2.546003,0.000000,0.000000,0.000000,2.072593,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.860565
2,0.000000,0.000000,0.000000,0.000000,0.122241,0.000658,0.007772,2.355508,0.607612,0.000000,0.000000,0.000000,2.293052,0.000000
3,0.020032,0.000000,0.000000,0.000000,0.014274,0.000000,0.023247,0.000000,1.229649,0.000000,3.663490,0.000000,0.000000,0.000000
4,0.000000,0.007493,0.000000,0.000000,0.000000,0.000000,7.152854,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.647747
5,0.000000,0.000000,0.000000,4.384445,0.000000,0.000621,0.000000,0.000000,0.000000,0.000000,0.050664,0.014213,0.088990,0.000000
6,0.081478,0.000000,2.057312,0.000358,0.055088,0.000060,0.006676,0.000000,0.184525,0.000000,0.000000,0.009614,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000265,0.000000,0.000000,0.000000,0.011028,0.061921,0.000000,0.000000,1.539088,0.000000,0.566813
8,0.000000,0.004982,0.000000,0.002600,0.000000,1.643795,0.000000,0.008665,0.000000,0.000000,0.000000,0.000000,0.000000,0.055784
9,0.082882,0.000000,0.000000,0.000000,0.000000,0.000343,0.000000,0.000000,0.000858,1.554218,0.000000,0.000000,0.032767,0.000000


In [30]:
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    print(f'For topic {topic+1} the words with the highest value are:')
    print(tmp.nlargest(10))
    print('\n')

For topic 1 the words with the highest value are:
co             8.395243
covid          0.211958
prices         0.071069
shopping       0.045082
need           0.001831
19             0.000000
consumer       0.000000
coronavirus    0.000000
covid19        0.000000
covid_19       0.000000
Name: 0, dtype: float64


For topic 2 the words with the highest value are:
19             2.546003
covid          2.072593
supermarket    0.860565
co             0.000000
consumer       0.000000
coronavirus    0.000000
covid19        0.000000
covid_19       0.000000
grocery        0.000000
like           0.000000
Name: 1, dtype: float64


For topic 3 the words with the highest value are:
grocery        2.355508
store          2.293052
like           0.607612
covid          0.122241
covid_19       0.007772
covid19        0.000658
19             0.000000
co             0.000000
consumer       0.000000
coronavirus    0.000000
Name: 2, dtype: float64


For topic 4 the words with the highest value are:
pr

## PCA

In [32]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['https','tco'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\liuru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
vect= TfidfVectorizer(stop_words=stop_words)
x = vect.fit_transform(tweets.OriginalTweet)
tf_idf_vect = pd.DataFrame(x.toarray().transpose(),index=vect.get_feature_names_out())


In [35]:
tf_idf_vect

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
0789,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
0kfsxfmzbh,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.25383,0.0,0.0,0.0,0.0,0.0,0.0
0pjxarzqlf,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yewlmjne9p,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
yogalesson,0.0,0.0,0.0,0.21283,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
youâ,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
zealand,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca.fit_transform(tf_idf_vect)

array([[ 0.00876471, -0.01628043,  0.03642314, ...,  0.00214661,
         0.01489529, -0.01080403],
       [-0.00588735, -0.00709817,  0.01107803, ..., -0.00395933,
         0.01434908,  0.03125605],
       [-0.01185696, -0.00849553,  0.01549713, ..., -0.00680164,
        -0.03437304, -0.00651578],
       ...,
       [-0.00705397,  0.03839263, -0.01966793, ..., -0.00876325,
        -0.02807194,  0.02202336],
       [ 0.00174348, -0.02192228,  0.03251023, ...,  0.00761634,
        -0.00650935,  0.00530256],
       [-0.02757536, -0.01312732, -0.00278709, ...,  0.00326623,
         0.00043568, -0.00743547]])

In [37]:
print(pca.components_)

[[-0.02843476  0.16574906  0.09886829 ... -0.00168628  0.2233878
   0.12914452]
 [-0.00385065 -0.09885568  0.04221106 ... -0.0136799  -0.07792497
  -0.04502291]
 [-0.00349224  0.13017387  0.05475532 ...  0.01556273 -0.22693506
  -0.03455713]
 ...
 [ 0.05567621  0.25322491 -0.08662793 ...  0.02191053 -0.04368965
  -0.08509353]
 [ 0.03463949  0.07290348  0.12361825 ... -0.0770317   0.08690375
   0.05504139]
 [ 0.10174061  0.09965147 -0.07024727 ... -0.07976001 -0.01633555
  -0.07582143]]


## SVD

In [39]:
import numpy as np

In [40]:
vect= TfidfVectorizer(stop_words=stop_words, smooth_idf=True)
x = vect.fit_transform(tweets.OriginalTweet).todense()
x = np.asarray(x)

In [41]:
from sklearn.decomposition import TruncatedSVD
svd_modeling = TruncatedSVD(n_components=4, algorithm='randomized', n_iter=100, random_state=122)
svd_modeling.fit(x)
components=svd_modeling.components_
vocab = vect.get_feature_names_out()

In [45]:
topic_word_list = []
def get_topics(components):
    for i, comp in enumerate(components):
        terms_comp =zip(vocab,comp)
        sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
        topic=" "
        for t in sorted_terms:
            topic= topic + ' ' + t[0]
        topic_word_list.append(topic)
        print(topic_word_list)
    return topic_word_list
get_topics(components)
        

['  co coronavirus 19 covid store grocery covid19']
['  co coronavirus 19 covid store grocery covid19', '  store grocery coronavirus covid19 employees co 4dnryvqfxb']
['  co coronavirus 19 covid store grocery covid19', '  store grocery coronavirus covid19 employees co 4dnryvqfxb', '  store grocery get employees covid seen go']
['  co coronavirus 19 covid store grocery covid19', '  store grocery coronavirus covid19 employees co 4dnryvqfxb', '  store grocery get employees covid seen go', '  sanitizer buy hand supermarket face people try']


['  co coronavirus 19 covid store grocery covid19',
 '  store grocery coronavirus covid19 employees co 4dnryvqfxb',
 '  store grocery get employees covid seen go',
 '  sanitizer buy hand supermarket face people try']